# Import libraries

In [1]:
import pyautogui
import gi
gi.require_version("Wnck", "3.0")
from gi.repository import Wnck

# TO DO : Run the game
``` bash
cd ./FlappyBirdGame
python ./flappy.py
```

# Get the game window datas

In [24]:
scr = Wnck.Screen.get_default()
scr.force_update()
windows = scr.get_windows()
FlappyBird_window = None
for window in windows:
     # if title is "Flappy Bird"
     if window.get_name() == "Flappy Bird":
        # get the window's geometry
        x, y, width, height = window.get_geometry()
        FlappyBird_window = { "x": x, "y": y, "width": width, "height": height }
        print("x :\t", x , "\ny :\t", y , "\nwidth :\t", width , "\nheight :", height)

x :	 40 
y :	 119 
width :	 288 
height : 549


In [31]:
FlappyBird_click = ( FlappyBird_window["x"] + FlappyBird_window["width"]/2, FlappyBird_window["y"] + FlappyBird_window["height"] - 30 )

# Some Tests

### Test screen capture

In [26]:
# make a screenshot of the window
img = pyautogui.screenshot(region=(FlappyBird_window["x"], FlappyBird_window["y"], FlappyBird_window["width"], FlappyBird_window["height"]))
# save the screenshot to a file
img.show()

### Test screen click

In [32]:
pyautogui.moveTo(FlappyBird_click)
pyautogui.click()
pyautogui.press('space')

# Build model

# Run the training session

# Tests and verifications

In [3]:
# pyautogui.confirm('Start AI?', buttons=['Go!'])

'Go!'